In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
%matplotlib inline

In [2]:
transaction = pd.read_csv('trans_new.csv')

In [3]:
transaction = transaction.drop(transaction.columns[0], axis = 1)
transaction.head()

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,696521494,211,55,5558,101111010,9907,2013-01-23,36.0,OZ,1,2.99
1,417112954,100,63,6320,107509070,182,2012-12-09,12.0,CT,1,2.29
2,308340717,360,35,3504,103500030,13470,2012-06-12,10.0,OZ,1,1.79
3,4629335121,153,63,6320,1068826767,81691,2012-09-10,12.0,CT,1,2.79
4,743051948,58,63,6330,101111010,9907,2012-10-07,16.0,OZ,1,2.79


In [4]:
transaction['date'] = transaction['date'].apply(lambda x: pd.to_datetime(x))

In [5]:
transaction.loc[transaction[transaction['productmeasure'] == '1'].index, 'productmeasure'] = 'OZ'

In [6]:
transaction['productmeasure'].fillna('Unknown',inplace = True)

In [7]:
transaction['price'] = transaction.apply(lambda x: x.purchaseamount/x.purchasequantity if (x.purchaseamount > 0 and x.purchasequantity > 0) else None, axis = 1)

In [8]:
transaction['year'] = transaction['date'].dt.year
transaction['month'] = transaction['date'].dt.month

In [9]:
transaction['dateblock'] = transaction.apply(lambda x: "lag_{}".format(3 - x.month) if x.year == 2013 else "lag_{}".format(15 - x.month), axis = 1)

In [10]:
data = transaction

### Build customer table

In [11]:
total_customer_transactions = pd.DataFrame({'id': data['id'].value_counts().index, 'No_of_transactions': data['id'].value_counts()})

In [12]:
total_customer_transactions.reset_index(drop = True, inplace = True)

In [13]:
feature_column = ['id','No_of_transactions']

In [14]:
total_customer_transactions = total_customer_transactions[feature_column]

In [15]:
total_customer_transactions.head()

,id,No_of_transactions
0,378964359,3590
1,618764932,2844
2,2255653295,2510
3,3465135195,1921
4,3288731255,1433


In [16]:
total_customer_quantity = data.groupby(by ='id', as_index = False)['purchasequantity'].sum()

In [17]:
total_customer_amount = data.groupby(by = 'id', as_index = False)['purchaseamount'].sum()

In [18]:
customer_table = pd.merge(total_customer_amount, total_customer_quantity, on = 'id')

In [19]:
customer_table = pd.merge(customer_table, total_customer_transactions, on = 'id')

In [20]:
customer_table.columns

Index([u'id', u'purchaseamount', u'purchasequantity', u'No_of_transactions'], dtype='object')

In [21]:
customer_table.columns = ['id','Amount_spent','Quantity_purchased','No_of_transactions']

In [22]:
customer_table.head()

,id,Amount_spent,Quantity_purchased,No_of_transactions
0,86246,57.06,14,12
1,86252,81.54,22,20
2,12277270,32.78,2,2
3,12332190,13.73,4,3
4,12524696,4.99,1,1


In [23]:
customer_table['Average_spent'] = customer_table.apply(lambda x: x.Amount_spent/x.Quantity_purchased if (x.Amount_spent > 0 and x.Quantity_purchased > 0) else 0, axis = 1)

In [24]:
customer_table.sort_values(by = ['Average_spent'], ascending = False).head(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent
145501,3370267887,1637.83,3,3,545.943333
74039,681840229,499.99,1,1,499.990000
78286,696165631,300.00,1,1,300.000000
139459,2977152560,288.75,1,1,288.750000
154774,3639216911,239.99,1,1,239.990000
64792,602345390,200.00,1,1,200.000000
109396,1969602481,188.98,1,1,188.980000
170882,4218554894,183.75,1,1,183.750000
101004,1782903910,506.58,3,3,168.860000
169511,4145487033,141.74,1,1,141.740000


In [25]:
customer_table.sort_values(by = ['Average_spent'], ascending = False).tail(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent
33280,474881726,-2.42,2,1,0.0
21890,400637151,-1.00,1,1,0.0
21894,400643354,-13.47,6,4,0.0
21898,400650139,-1.00,1,1,0.0
57204,565544037,0.00,1,1,0.0
70976,658200738,0.00,1,1,0.0
122892,2517971595,0.00,1,1,0.0
21905,400658541,-8.03,2,1,0.0
157997,3716494508,0.00,1,1,0.0
83562,716828270,-1.00,1,1,0.0


In [26]:
len(customer_table[customer_table['Average_spent'] == 0])

4425

In [27]:
len(customer_table)

193779

In [28]:
customer_table.sort_values(by = ['Quantity_purchased'], ascending = False).head(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent
20798,378964359,76745.83,29233,3590,2.625315
148559,3465135195,37373.02,14192,1921,2.633386
68764,618764932,38226.87,12662,2844,3.019023
114432,2255653295,34561.73,10071,2510,3.431807
143505,3288731255,13087.07,6449,1433,2.029318
146232,3388824436,12705.44,4383,999,2.898800
146231,3388822242,16616.18,2869,760,5.791628
146228,3388820702,4969.34,2598,593,1.912756
152341,3594433805,6695.07,2386,869,2.805981
19934,347580535,5419.24,1650,765,3.284388


In [29]:
customer_table.sort_values(by = ['Quantity_purchased'], ascending = False).tail(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent
104547,1857543780,-2.97,-3,1,0.0
168129,4099872494,-4.00,-3,1,0.0
51801,533849214,-5.25,-3,1,0.0
90073,742086371,-5.30,-3,1,0.0
34558,479073527,-16.76,-4,1,0.0
98339,1345718776,1.78,-4,2,0.0
39483,496157126,-1.40,-6,1,0.0
75242,684542088,-30.12,-8,2,0.0
23031,404539539,3.84,-8,5,0.0
117487,2340802534,10.00,-33,1,0.0


In [30]:
earliest_order = data.groupby(by = 'id', as_index = False)['date'].min()

In [31]:
earliest_order.columns = ['id','Earliest_transaction']

In [32]:
earliest_order['now'] = data['date'].max()

In [33]:
earliest_order.iloc[0]['now']-earliest_order.iloc[0]['Earliest_transaction']

Timedelta('329 days 00:00:00')

In [34]:
(earliest_order.iloc[1]['now']-earliest_order.iloc[1]['Earliest_transaction']).days

394

In [35]:
days = []
for i in range(len(earliest_order)):
    days.append(1+(earliest_order.iloc[i]['now']-earliest_order.iloc[i]['Earliest_transaction']).days)
earliest_order['days_as_customer'] = days

In [36]:
earliest_order.drop('now', axis = 1, inplace = True)

In [37]:
latest_order = data.groupby(by = 'id', as_index = False)['date'].max()

In [38]:
latest_order.columns = ['id','Latest_transaction']

In [39]:
latest_order['now'] = data['date'].max()

In [40]:
days = []
for i in range(len(latest_order)):
    days.append(1+(latest_order.iloc[i]['now']-latest_order.iloc[i]['Latest_transaction']).days)
latest_order['days_since_purchase'] = days

In [41]:
latest_order.drop('now',axis = 1, inplace = True)

In [42]:
earliest_order.head()

,id,Earliest_transaction,days_as_customer
0,86246,2012-05-06,330
1,86252,2012-03-02,395
2,12277270,2012-09-07,206
3,12332190,2012-06-15,290
4,12524696,2012-12-06,116


In [43]:
latest_order.head()

,id,Latest_transaction,days_since_purchase
0,86246,2013-03-30,2
1,86252,2013-03-25,7
2,12277270,2012-11-14,138
3,12332190,2012-10-02,181
4,12524696,2012-12-06,116


In [44]:
customer_table = pd.merge(customer_table, earliest_order, on = 'id')

In [45]:
customer_table = pd.merge(customer_table, latest_order, on = 'id')

In [46]:
customer_table.head(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent,Earliest_transaction,days_as_customer,Latest_transaction,days_since_purchase
0,86246,57.06,14,12,4.075714,2012-05-06,330,2013-03-30,2
1,86252,81.54,22,20,3.706364,2012-03-02,395,2013-03-25,7
2,12277270,32.78,2,2,16.390000,2012-09-07,206,2012-11-14,138
3,12332190,13.73,4,3,3.432500,2012-06-15,290,2012-10-02,181
4,12524696,4.99,1,1,4.990000,2012-12-06,116,2012-12-06,116
5,12682470,1.68,1,1,1.680000,2013-02-17,43,2013-02-17,43
6,13074629,21.31,7,4,3.044286,2012-11-06,146,2013-01-26,65
7,13089312,10.47,4,4,2.617500,2012-06-23,282,2013-02-07,53
8,13179265,7.47,3,2,2.490000,2012-03-02,395,2012-09-09,204
9,13251776,11.99,2,2,5.995000,2012-09-28,185,2012-11-06,146


##        There are at least two ways we can build customer segmentation models. 
##      First I want to introduce the standard RFM approach.

In [47]:
customer_table.describe()

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent,days_as_customer,days_since_purchase
count,1.937790e+05,193779.000000,193779.000000,193779.000000,193779.000000,193779.000000,193779.000000
mean,1.891321e+09,10.790846,3.742629,2.323534,3.285876,238.498950,140.442344
std,1.558134e+09,239.182019,86.228317,15.205789,3.862094,111.108547,105.455680
min,8.624600e+04,-108.120000,-33.000000,1.000000,0.000000,1.000000,1.000000
25%,5.001740e+08,3.150000,1.000000,1.000000,1.830000,149.000000,52.000000
50%,9.839479e+08,6.040000,2.000000,2.000000,2.717500,258.000000,116.000000
75%,3.365842e+09,11.325000,4.000000,3.000000,3.867500,336.000000,213.000000
max,4.808333e+09,76745.830000,29233.000000,3590.000000,545.943333,395.000000,395.000000


In [48]:
def big_spender(row):
    if row['Amount_spent'] >= 11.325:
        return 1
    elif row['Amount_spent'] >=6.04:
        return 2
    elif row['Amount_spent'] >=3.15:
        return 3
    else:
        return 4

In [49]:
def many_purchase(row):
    if row['Quantity_purchased'] >= 4:
        return 1
    elif row['Quantity_purchased'] >= 2:
        return 2
    elif row['Quantity_purchased'] >= 1:
        return 3
    else:
        return 4

In [50]:
def many_orders(row):
    if row['No_of_transactions'] >= 3:
        return 1
    elif row['No_of_transactions'] >= 2:
        return 2
    elif row['No_of_transactions'] >= 1:
        return 3
    

In [51]:
def loyal_customer(row):
    if row['days_as_customer'] >= 335:
        return 1
    elif row['days_as_customer'] >= 257:
        return 2
    elif row['days_as_customer'] >= 148:
        return 3
    else:
        return 4

In [52]:
def dormant_customer(row):
    if row['days_since_purchase'] >= 212:
        return 1
    elif row['days_since_purchase'] >= 115:
        return 2
    elif row['days_since_purchase'] >= 51:
        return 3
    else:
        return 4      

In [53]:
customer_table['Order_frequency'] = customer_table['No_of_transactions']/customer_table['days_as_customer']

In [54]:
purchase_freq = []
for i in range(len(customer_table)):
    if customer_table.iloc[i]['Quantity_purchased'] > 0:
       purchase_freq.append(customer_table.iloc[i]['Quantity_purchased']/float(customer_table.iloc[i]['days_as_customer']))
    else:
        purchase_freq.append(0)
customer_table['Purchase_frequency'] = purchase_freq

In [55]:
customer_table['Big_spender'] = customer_table.apply(big_spender, axis = 1)
customer_table['Many_purchase'] = customer_table.apply(many_purchase, axis = 1)
customer_table['Many_orders'] = customer_table.apply(many_orders, axis = 1)
customer_table['Loyal_customer'] = customer_table.apply(loyal_customer, axis = 1)
customer_table['Dormant_customer'] = customer_table.apply(dormant_customer, axis = 1)

In [56]:
customer_table.head(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent,Earliest_transaction,days_as_customer,Latest_transaction,days_since_purchase,Order_frequency,Purchase_frequency,Big_spender,Many_purchase,Many_orders,Loyal_customer,Dormant_customer
0,86246,57.06,14,12,4.075714,2012-05-06,330,2013-03-30,2,0.036364,0.042424,1,1,1,2,4
1,86252,81.54,22,20,3.706364,2012-03-02,395,2013-03-25,7,0.050633,0.055696,1,1,1,1,4
2,12277270,32.78,2,2,16.390000,2012-09-07,206,2012-11-14,138,0.009709,0.009709,1,2,2,3,2
3,12332190,13.73,4,3,3.432500,2012-06-15,290,2012-10-02,181,0.010345,0.013793,1,1,1,2,2
4,12524696,4.99,1,1,4.990000,2012-12-06,116,2012-12-06,116,0.008621,0.008621,3,3,3,4,2
5,12682470,1.68,1,1,1.680000,2013-02-17,43,2013-02-17,43,0.023256,0.023256,4,3,3,4,4
6,13074629,21.31,7,4,3.044286,2012-11-06,146,2013-01-26,65,0.027397,0.047945,1,1,1,4,3
7,13089312,10.47,4,4,2.617500,2012-06-23,282,2013-02-07,53,0.014184,0.014184,2,1,1,2,3
8,13179265,7.47,3,2,2.490000,2012-03-02,395,2012-09-09,204,0.005063,0.007595,2,2,2,1,2
9,13251776,11.99,2,2,5.995000,2012-09-28,185,2012-11-06,146,0.010811,0.010811,1,2,2,3,2


In [57]:
customer_table.sort_values(by=['Big_spender','Purchase_frequency','Many_purchase','Many_orders','Dormant_customer','Loyal_customer'], ascending = [True, False, True, True, False,True]).head(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent,Earliest_transaction,days_as_customer,Latest_transaction,days_since_purchase,Order_frequency,Purchase_frequency,Big_spender,Many_purchase,Many_orders,Loyal_customer,Dormant_customer
20798,378964359,76745.83,29233,3590,2.625315,2012-03-02,395,2013-03-31,1,9.088608,74.007595,1,1,1,1,4
148559,3465135195,37373.02,14192,1921,2.633386,2012-03-02,395,2013-03-24,8,4.863291,35.929114,1,1,1,1,4
68764,618764932,38226.87,12662,2844,3.019023,2012-03-02,395,2013-03-18,14,7.200000,32.055696,1,1,1,1,4
114432,2255653295,34561.73,10071,2510,3.431807,2012-03-02,395,2013-03-31,1,6.354430,25.496203,1,1,1,1,4
143505,3288731255,13087.07,6449,1433,2.029318,2012-03-02,395,2013-03-14,18,3.627848,16.326582,1,1,1,1,4
146232,3388824436,12705.44,4383,999,2.898800,2012-03-02,395,2013-03-16,16,2.529114,11.096203,1,1,1,1,4
22278,401342615,21.68,1211,1,0.017903,2012-11-19,133,2012-11-19,133,0.007519,9.105263,1,1,3,4,2
146231,3388822242,16616.18,2869,760,5.791628,2012-03-02,395,2013-03-12,20,1.924051,7.263291,1,1,1,1,4
174772,4427711419,2339.78,768,380,3.046589,2012-12-07,115,2013-03-31,1,3.304348,6.678261,1,1,1,4,4
146228,3388820702,4969.34,2598,593,1.912756,2012-03-02,395,2013-03-23,9,1.501266,6.577215,1,1,1,1,4


In [59]:
customer_table.sort_values(by=['Big_spender','Purchase_frequency','Many_purchase','Many_orders','Dormant_customer','Loyal_customer'], ascending = [True, False, True, True, False,True]).tail(10)

,id,Amount_spent,Quantity_purchased,No_of_transactions,Average_spent,Earliest_transaction,days_as_customer,Latest_transaction,days_since_purchase,Order_frequency,Purchase_frequency,Big_spender,Many_purchase,Many_orders,Loyal_customer,Dormant_customer
123010,2521089521,0.00,0,1,0.0,2012-08-07,237,2012-08-07,237,0.004219,0.0,4,4,3,3,1
134542,2813229545,-3.00,-1,1,0.0,2012-07-29,246,2012-07-29,246,0.004065,0.0,4,4,3,3,1
140732,3010828419,0.00,0,1,0.0,2012-08-15,229,2012-08-15,229,0.004367,0.0,4,4,3,3,1
141952,3043233717,-0.79,0,1,0.0,2012-09-01,212,2012-09-01,212,0.004717,0.0,4,4,3,3,1
141975,3043977103,0.00,0,1,0.0,2012-08-07,237,2012-08-07,237,0.004219,0.0,4,4,3,3,1
154818,3639861299,0.00,0,1,0.0,2012-08-11,233,2012-08-11,233,0.004292,0.0,4,4,3,3,1
164542,3975549503,0.00,0,1,0.0,2012-08-23,221,2012-08-23,221,0.004525,0.0,4,4,3,3,1
165278,4001876758,0.00,0,1,0.0,2012-08-30,214,2012-08-30,214,0.004673,0.0,4,4,3,3,1
169660,4150471198,0.00,0,1,0.0,2012-07-26,249,2012-07-26,249,0.004016,0.0,4,4,3,3,1
189480,4636237439,0.00,0,1,0.0,2012-07-21,254,2012-07-21,254,0.003937,0.0,4,4,3,3,1


In [60]:
customer_table.Purchase_frequency.describe()

count    193779.000000
mean          0.023037
std           0.345363
min           0.000000
25%           0.006329
50%           0.010989
75%           0.019022
max         113.000000
Name: Purchase_frequency, dtype: float64

In [61]:
def high_frequency(row):
    if row['Purchase_frequency'] >= 0.019022 :
        return 1
    elif row['Purchase_frequency'] >= 0.010989:
        return 2
    elif row['Purchase_frequency'] >= 0.006329 :
        return 3
    else:
        return 4

In [62]:
customer_table['High_frequency'] = customer_table.apply(high_frequency, axis = 1)

### Premium customer criteria: 
### Big_spend = 1, High_frequency = 1, Many_purchase = 1, Dormant_customer = 4, Loyal_customer = 1 

In [63]:
len(customer_table[(customer_table['Big_spender'] == 1) &(customer_table['High_frequency'] == 1) &(customer_table['Many_purchase'] == 1) & (customer_table['Dormant_customer'] == 4) &(customer_table['Loyal_customer'] == 1)])

2768

In [64]:
2768.0 / 193779   #only one percent satisfies the above premimum customer criteria

0.014284313573710258

### Random customer criteria:
### Big_spend = 4, High_frequency = 4, Many_purchase = 4, Dormant_customer = 1, Loyal_customer = 3

In [65]:
## customers who do not purchase anything have the lowest level of purchase frequency
len(customer_table[(customer_table['Big_spender'] == 4) &(customer_table['High_frequency'] == 4) &(customer_table['Many_purchase'] == 4) & (customer_table['Dormant_customer'] == 1) &(customer_table['Loyal_customer'] == 3)])

23

In [66]:
# customers who purchase only one item
len(customer_table[(customer_table['Big_spender'] == 4)  &(customer_table['Many_purchase'] == 3) & (customer_table['Dormant_customer'] == 1) &(customer_table['Loyal_customer'] == 3)])

3861

In [67]:
len(customer_table[(customer_table['Big_spender'] == 4)  &(customer_table['Many_purchase'] == 4) & (customer_table['Dormant_customer'] == 1) &(customer_table['Loyal_customer'] == 3)])

23

In [68]:
# customers who purchase only one item have the lowest level of purchase frequency
len(customer_table[(customer_table['Big_spender'] == 4) &(customer_table['High_frequency'] == 4) &(customer_table['Many_purchase'] == 3) & (customer_table['Dormant_customer'] == 1) &(customer_table['Loyal_customer'] == 3)])

3861